Code scraping fonctionnel

In [7]:
import pandas as pd
df1 = pd.read_csv('/Users/camille/repo/Hetic/projet_gouv/scraping/Data/rec.csv', sep=';')

In [9]:
import time
import random
import re
import os
import logging
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys


DRIVER_PATH = r'/home/oli/Projects/Google-review-scraper/chromedriver_linux64/chromedriver'
SAVING_PATH = '/Users/camille/repo/Hetic/projet_gouv/scraping/Data'

# declaring a list, that contains the urls wich we want to be scraped
OBJECT_URLS = "https://www.google.com/maps/"
    

# setting up the logging object
logger = logging.getLogger('main')
logging.basicConfig(
    format='[%(asctime)s] [%(levelname)s] - %(message)s',
    datefmt='%H:%M:%S'
    )

# we can change the logging level. Use logging.DEBUG if necesarry
logger.setLevel(logging.DEBUG)


def scrape_an_object(object_url, location):
    # setting the chrome driver for selenium
    driver = webdriver.Chrome(service=Service(DRIVER_PATH))

    # opening the given URL
    logger.debug("Opening the given URL")
    driver.get(object_url)

    # accepting the cookies
    logger.debug("Accepting the cookies")
    driver.find_element(By.CLASS_NAME,"lssxud").click()

    # waiting some random seconds
    time.sleep(random.uniform(4,6))
    select_box = driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
    select_box.send_keys(location)
    select_box.send_keys(Keys.ENTER)
    time.sleep(2)
    object_name = driver.find_element(
    By.CSS_SELECTOR,
    'h1.DUwDvf.fontHeadlineLarge'
    ).text
    logger.debug(f'Object_name OK : {object_name}')

    object_address = driver.find_element(
        By.CSS_SELECTOR,
        'div.Io6YTe.fontBodyMedium'
    ).text
    logger.debug(f'Object_address OK : {object_address}')

    # I use CSS selectors where I can, because its more robust than XPATH



    try:

        overall_rating = driver.find_element(
            By.CSS_SELECTOR,
            'div.F7nice.mmu3tf'
        ).text.split()[0]
        logger.debug(f'Overall_rating OK : {overall_rating}')

        review_number = driver.find_element(
            By.CSS_SELECTOR,
            'div.F7nice.mmu3tf'
        ).text.replace(' ','')

        review_number = int(re.compile(r'\d+').findall(review_number)[-1])
        logger.debug(f'Review_number OK : {review_number}')

        # click to load further reviews
        driver.find_element(
            By.XPATH,
            '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span[2]/span[1]/span'
        ).click()

        logger.debug('Clicked to load further reviews')
    
        time.sleep(random.uniform(0.1, 0.5))

        # find scroll layout
        scrollable_div = driver.find_element(
            By.XPATH,
            '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
    

        logger.debug('Scroll div OK')


    except NoSuchElementException:

        logger.debug('Except branch')

        div_num_rating = driver.find_element(
            By.CSS_SELECTOR,
            'div.F7nice'
        ).text
        overall_rating = div_num_rating.split()[0]
        logger.debug(f'Overall_rating OK : {overall_rating}')

        review_number = int(div_num_rating.split()[1].replace('(','').replace(')',''))
        logger.debug(f'Review_number OK : {review_number}')

        # click on the review tab
        driver.find_element(By.XPATH,'/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[1]').click()
        logger.debug('clicked to load further reviews')

        time.sleep(random.uniform(0.1, 0.5))

        # find scroll layout
        scrollable_div = driver.find_element(
            By.XPATH,
            '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]'
        )
        logger.debug('Scroll div OK')
        #button = driver.element_to_be_clickable((By.CSS_SELECTOR, "button.w8nwRe"))
        #button.click()


    time.sleep(random.uniform(2,4))

    # scroll as many times as necessary to load all reviews
    for _ in range(0,(round(review_number/5 - 1)+1)):
        driver.execute_script(
            'arguments[0].scrollTop = arguments[0].scrollHeight',
            scrollable_div
        )
        # click on 'more' botton if it appears
        try:
            button = driver.find_element(
                By.CSS_SELECTOR,
                'button.w8nwRe.kyuRq'
            )
            button.click()
        except: 
            pass
        time.sleep(random.uniform(1, 2))

    # parse the html with a bs object
    response = BeautifulSoup(driver.page_source, 'html.parser')
    reviews_source = response.find_all('div', class_='jJc9Ad')
    logger.debug('Source code has been parsed!')

    # closing the browser
    
        
    

    # storing the data in a dict
    store_main_data = {'object_name': object_name,
                       'object_address': object_address,
                       'overall_rating': overall_rating,
                       'review_num': review_number,
                       'object_url':object_url}

    return store_main_data, reviews_source

def extract_reviews(reviews_source: list) -> list:

    r"""
    This method processes the input html code and returns a list 
    containing the reviews.

    """

    review_list = []

    logger.debug('Starting iterate trough the reviews...')
    for review in reviews_source:

        # extract the relevant informations
        user = review.find('div', class_= 'd4r55').text.strip()
        date = review.find('span', class_= 'rsqaWe').text.strip()
        rate = len(review.find('span',class_ = 'kvMYJc'))
        review_text = review.find('span', class_= 'wiI7pd')
        review_text = '' if review_text is None else review_text.text 
        reply_source = review.find('div', class_= 'CDe7pd')
        reply = reply_source.text if reply_source else '-'


        review_list.append({'name': user,
                            'date': date,
                            'rate': rate,
                            'review_text': review_text,
                            'reply': reply})

    return review_list




def main():

    # creating a list to store all objects scraped
    all_objects_data = []

    # iterating through each object url in the list
    for location in df1['GM_search']:
        store_main_data, reviews_source = scrape_an_object(OBJECT_URLS, location)
        store_main_data = extract_reviews(reviews_source)
        all_objects_data.append(store_main_data)
        logger.debug(f'{location} is done!')

    # creating a dataframe from the list of dictionaries
    df = pd.DataFrame(all_objects_data)

    # writing the dataframe to a csv file
    df.to_csv(os.path.join(SAVING_PATH, 'google_reviews.csv'), index=False)

if __name__ == '__main__':
    main()



[00:10:50] [DEBUG] - Opening the given URL
[00:10:50] [DEBUG] - Accepting the cookies
[00:10:58] [DEBUG] - Object_name OK : Rectorat de l'académie de Lille
[00:10:58] [DEBUG] - Object_address OK : 144 Rue de Bavay, 59000 Lille
[00:10:58] [DEBUG] - Except branch
[00:10:58] [DEBUG] - Overall_rating OK : 1.9
[00:10:58] [DEBUG] - Review_number OK : 202
[00:10:58] [DEBUG] - clicked to load further reviews
[00:10:58] [DEBUG] - Scroll div OK


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=113.0.5672.126)
Stacktrace:
0   chromedriver                        0x00000001053178ac chromedriver + 4257964
1   chromedriver                        0x000000010530ff40 chromedriver + 4226880
2   chromedriver                        0x0000000104f4c9d4 chromedriver + 281044
3   chromedriver                        0x0000000104f26db0 chromedriver + 126384
4   chromedriver                        0x0000000104fab938 chromedriver + 670008
5   chromedriver                        0x0000000104fbdfe8 chromedriver + 745448
6   chromedriver                        0x0000000104f7b98c chromedriver + 473484
7   chromedriver                        0x0000000104f7c98c chromedriver + 477580
8   chromedriver                        0x00000001052d6900 chromedriver + 3991808
9   chromedriver                        0x00000001052da354 chromedriver + 4006740
10  chromedriver  